# Use Gen AI to build DENG pipeline 

This presents the idea for how to use GenAI, to help to build a medallion architecture

## Basic Setup
We'll use [duckdb](https://duckdb.org/) as the database.

In [1]:
import duckdb
# import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Config
# %config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True
%config SqlMagic.displaylimit = None

# Connect to DuckDB
# %sql duckdb:///:default:
# %sql duckdb:///:memory:
%sql duckdb:////Users/jesse23/u/p/tide/notes/test.db

displaylimit: Value None will be treated as 0 (no limit)

Assuming we have source data as below:

In [2]:
%%sql 
-- part data, assuming import by a CSV
CREATE TABLE IF NOT EXISTS bronze_design
  (id INTEGER, name VARCHAR, type VARCHAR, designer VARCHAR);
INSERT INTO bronze_design VALUES 
  (0,  'part0',   'part', 'Allen'), 
  (1,  'part1',   'part', 'Bobby'), 
  (2,  'part2',   'part', 'Cindy'), 
  (3,  'part00',  'part', 'David'), 
  (4,  'part001', 'part', 'Eddie'), 
  (5,  'part002', 'part', 'Frank'), 
  (6,  'part10',  'part', 'George'), 
  (7,  'part100', 'part', 'Harry'), 
  (8,  'part101', 'part', 'Ivan'), 
  (9,  'part11',  'part', 'Jack'), 
  (10, 'part110', 'part', 'Kevin'), 
  (11, 'part111', 'part', 'Larry'); 

-- company info, import by a CSV
CREATE TABLE IF NOT EXISTS bronze_design_office
  (design VARCHAR, office VARCHAR);
INSERT INTO bronze_design_office VALUES 
  ('part001', 'SmartView' ), 
  ('part10', 'FutureIdea'); 

SELECT * from bronze_design;

id,name,type,designer
0,part0,part,Allen
1,part1,part,Bobby
2,part2,part,Cindy
3,part00,part,David
4,part001,part,Eddie
5,part002,part,Frank
6,part10,part,George
7,part100,part,Harry
8,part101,part,Ivan
9,part11,part,Jack


In [11]:
%%sql
CREATE TABLE IF NOT EXISTS silver_design
(
    uuid UUID PRIMARY KEY,
    name VARCHAR,
    type VARCHAR,
    designer VARCHAR,
    parent VARCHAR,
    group_name VARCHAR,
    office VARCHAR
);
WITH bronze_design_augmented AS (
    SELECT 
        *,
        uuid() AS uuid,
        -- LENGTH(name) - LENGTH(REPLACE(name, '1', '')) AS level
    FROM 
        bronze_design
),
design_hierarchy AS (
    SELECT 
        bd1.uuid,
        bd1.id,
        bd1.name,
        bd1.type,
        bd1.designer,
        -- bd1.level,
        bd2.name AS parent
    FROM 
        bronze_design_augmented bd1
    LEFT JOIN 
        bronze_design_augmented bd2
    ON 
        bd2.name = SUBSTRING(bd1.name FROM 1 FOR LENGTH(bd1.name) - 1) 
        -- AND bd1.level = bd2.level + 1
),
design_groups AS (
    SELECT 
        dh1.uuid,
        dh1.name,
        dh1.type,
        dh1.designer,
        dh1.parent,
        COALESCE(dh2.name, dh1.name) AS group_name
    FROM 
        design_hierarchy dh1
    LEFT JOIN 
        design_hierarchy dh2
    ON 
        dh1.parent = dh2.name
),
design_offices AS (
    SELECT 
        dg.uuid,
        dg.name,
        dg.type,
        dg.designer,
        dg.parent,
        dg.group_name,
        COALESCE(bdo.office, '') AS office
    FROM 
        design_groups dg
    LEFT JOIN 
        bronze_design_office bdo
    ON 
        dg.name = bdo.design OR dg.group_name = bdo.design
)
INSERT INTO silver_design
SELECT 
    uuid,
    name,
    type,
    designer,
    parent,
    group_name,
    office
FROM 
    design_offices;



Count
12


In [9]:
%%sql
DELETE from silver_design;

Count


In [12]:
%%sql
select * from silver_design;

uuid,name,type,designer,parent,group_name,office
60a8c7e3-5c03-4ae3-8d47-b14f72fa8d13,part001,part,Eddie,part00,part00,SmartView
91313fff-f932-4db5-94fd-945c784c1f6a,part10,part,George,part1,part1,FutureIdea
2eaf75f6-9bb5-4879-b4a6-890768304f7d,part100,part,Harry,part10,part10,FutureIdea
487932c9-3db8-410f-a9af-e607718e6b0f,part101,part,Ivan,part10,part10,FutureIdea
fccc42a6-3a14-4d04-b73d-05c062546262,part00,part,David,part0,part0,
f67fdd06-b06e-47ac-85cb-aea133433302,part002,part,Frank,part00,part00,
42fbe36a-1bdd-495c-8072-4302d5d90e23,part11,part,Jack,part1,part1,
826dd463-024f-425a-bc7d-7f29a1716963,part110,part,Kevin,part11,part11,
5ad3927f-063d-47c5-8905-eed638024e7a,part111,part,Larry,part11,part11,
3e3bd427-5a04-444a-bbc1-218d65000c07,part0,part,Allen,None,part0,
